## Duplicate checks

In this recipe we are going to run duplicate checks.  
Note that this is not about "fuzzy" i.e. "approximate" duplicates but actual duplications in the data.  
These can have various causes and what we may want to do with them, to choose the best approach:  

a) very low number of duplicates due to anomalies in the data itself or (more likely) errors/limitations in the SQL used for extracting data.  
For example, there could be a rare transaction settled with multiple currrencies and the extraction (SQL joins used) generated multiple rows, one per each currency  
In this case we probably want to know them and export them somewhere for investigation, but we may decide to exclude these transactions from further aggregation.  
b) Another example is when "stitching" together extractions and we get overlaps accidentally. It could be due to errors in the date ranges, or sometimes it is just due to changes in the data itself. E.g. we extract first all active/open transactions, then the closed ones and while we were generating the queries, there were transactions closing in the live system, so we would have these twice.  
c) In many cases it is just overload of SQL joins. For example IT provides us a fully joined ("merging" in Pandas lingo) header, lines and maybe other downstream data, so , a variation of a) but with lots of spurious duplications.  
Longer example: we get supplier invoices with header, lines, GL account and information on when we may have done (multiple) payments. All in one file. Good luck with that.  
In this case we tend to try to create "clean" versions, so the duplicate count becomes a bit pointless and we want to pick the first/unique instances of the headers and line fields separately,  or use some sort of aggregation to keep counts  
 





In [9]:

import pandas as pd
import numpy as np


https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html

In [10]:
demo_data={'col1':[1,2,3,3,5,6,6,7,7,np.nan,np.nan],'col2':['a','b','b','b','c','e','f','f','f','h','i'], 'col3':[1,1,2,2,1,1,1,1,2,1,1]}
df1= pd.DataFrame(demo_data)
df1

,col1,col2,col3
0,1.0,a,1
1,2.0,b,1
2,3.0,b,2
3,3.0,b,2
4,5.0,c,1
5,6.0,e,1
6,6.0,f,1
7,7.0,f,1
8,7.0,f,2
9,NaN,h,1


In [11]:
df1.sort_values(["col1","col3"], ascending=[True,False], inplace=True)
df1

,col1,col2,col3
0,1.0,a,1
1,2.0,b,1
2,3.0,b,2
3,3.0,b,2
4,5.0,c,1
5,6.0,e,1
6,6.0,f,1
8,7.0,f,2
7,7.0,f,1
9,NaN,h,1


In [12]:
df1.drop_duplicates()

,col1,col2,col3
0,1.0,a,1
1,2.0,b,1
2,3.0,b,2
4,5.0,c,1
5,6.0,e,1
6,6.0,f,1
8,7.0,f,2
7,7.0,f,1
9,NaN,h,1
10,NaN,i,1


In [13]:
df1.drop_duplicates(subset=['col1'])

,col1,col2,col3
0,1.0,a,1
1,2.0,b,1
2,3.0,b,2
4,5.0,c,1
5,6.0,e,1
8,7.0,f,2
9,NaN,h,1


In [14]:
df1.duplicated(subset=['col1']) 
# note this yields a list of True/False (boolean values) telling us
# whether a value is duplicated as per the criteria we provided
# so we can easily add as a separate column for future filtering.

0     False
1     False
2     False
3      True
4     False
5     False
6      True
8     False
7      True
9     False
10     True
dtype: bool

In [15]:
df1['dupes_full_record']=df1.duplicated(keep=False)
df1['dupes_col1'] =df1.duplicated(subset=["col1"], keep=False)
df1['dupes_col1_col2'] =df1.duplicated(subset=["col1","col2"], keep=False)

df1


,col1,col2,col3,dupes_full_record,dupes_col1,dupes_col1_col2
0,1.0,a,1,False,False,False
1,2.0,b,1,False,False,False
2,3.0,b,2,True,True,True
3,3.0,b,2,True,True,True
4,5.0,c,1,False,False,False
5,6.0,e,1,False,True,False
6,6.0,f,1,False,True,False
8,7.0,f,2,False,True,True
7,7.0,f,1,False,True,True
9,NaN,h,1,False,True,False


Interesting points to note:  
a) NAN is picked as duplicates if there are more than one empty one, otherwise it doesnt.  
b) you can put a "subset" parameter to set which fields to consider or you could do it by defining the df[['field_name_1','field_name_2]].duplicated() approach. The former is better if you wish to retrieve the entire record, while the later just deals with that selected dataframe and may be good just for a quick check of duplications.
c) we created flag columns but you can just use them as a filter as df[ df.duplicated()].  





In [16]:
df1[df1.duplicated(keep=False)]

,col1,col2,col3,dupes_full_record,dupes_col1,dupes_col1_col2
2,3.0,b,2,True,True,True
3,3.0,b,2,True,True,True


In [17]:
df1[df1.duplicated(subset=['col1','col2'],keep=False)]

,col1,col2,col3,dupes_full_record,dupes_col1,dupes_col1_col2
2,3.0,b,2,True,True,True
3,3.0,b,2,True,True,True
8,7.0,f,2,False,True,True
7,7.0,f,1,False,True,True


This is the basic command to select the duplicates.  
We are telling python to retrieve the dataframe records for which there are duplicated customer_id, keeping all of the duplicates ("keep" can be used also to get only the first or last instance)

In [18]:
df = pd.read_excel("./input/data.xlsx", sheet_name="Sheet1")
print(df.shape)  # "Shape" shows count of columns and rows
df.head()



(933, 9)


,customer_id,gender,first_name,last_name,dob,date_enrollment,city,country,residential address
0,1,M,Chad,Rios,2016-10-05,2013-03-10,Port Rogerhaven,Bahamas,8525 Emily Bypass
1,2,F,Rachel,Lynch,2011-11-09,2020-01-04,Thompsonbury,Cook Islands,344 Nicholas Bypass Suite 895
2,3,M,John,Barber,2001-11-05,2017-06-11,North Christine,Turkmenistan,3914 Rodney Crest Apt. 837
3,4,M,David,Lewis,2015-01-22,2006-09-13,Port Anthony,Bouvet Island (Bouvetoya),901 Alexis Hill
4,5,M,Michael,Webster,2015-05-26,2008-05-18,South Miguel,Yemen,74108 Stephanie Shore


In [19]:
len(df.drop_duplicates()) #

933

 we have some full record duplications  
 remember that the commands by default doesnt affect dataframe. You need to assign to variable or use inplace=True

In [20]:
df.drop_duplicates(['customer_id','first_name']) # looking just at two column, that is allowing some fuzziness regarding the rest.

,customer_id,gender,first_name,last_name,dob,date_enrollment,city,country,residential address
0,1,M,Chad,Rios,2016-10-05,2013-03-10,Port Rogerhaven,Bahamas,8525 Emily Bypass
1,2,F,Rachel,Lynch,2011-11-09,2020-01-04,Thompsonbury,Cook Islands,344 Nicholas Bypass Suite 895
2,3,M,John,Barber,2001-11-05,2017-06-11,North Christine,Turkmenistan,3914 Rodney Crest Apt. 837
3,4,M,David,Lewis,2015-01-22,2006-09-13,Port Anthony,Bouvet Island (Bouvetoya),901 Alexis Hill
4,5,M,Michael,Webster,2015-05-26,2008-05-18,South Miguel,Yemen,74108 Stephanie Shore
...,...,...,...,...,...,...,...,...,...
928,996,F,Theresa,Newman,1996-09-08,2021-06-30,Murphyhaven,Monaco,19698 Mclean Viaduct Apt. 470
929,997,F,Julia,Shaw,2020-05-17,2003-05-14,Dominicmouth,Zimbabwe,78054 Shawn Extension Suite 533
930,998,F,Sherry,Russo,2004-01-21,2001-10-07,West Jamesside,Nepal,7484 Rodriguez Rapid Apt. 845
931,999,F,Mackenzie,Berg,2013-11-29,2003-04-24,Roseburgh,Saint Vincent and the Grenadines,9494 Woods Highway Suite 210


In [21]:
#checking how many custimer_id duplicates are
df[df.duplicated(['customer_id'], keep=False)].shape

(0, 9)

In [22]:
df_duplicates = df[df.duplicated(['customer_id'],keep=False)]
df_without_duplicates= df.drop_duplicates(['customer_id'], keep=False) # this is without the duplicates completely
df_only_first_occurrence= df.drop_duplicates(['customer_id'], keep="first") # this is the first instance, you need to sort first if you are interest in cronological or largest etc.
# df_duplicates.to_excel("./output/102_Duplicates.xlsx", sheet_name='Duplicates', index=False)

In [23]:
df.drop_duplicates(['customer_id'], keep='first') # keep can be first, last or False, default is first

,customer_id,gender,first_name,last_name,dob,date_enrollment,city,country,residential address
0,1,M,Chad,Rios,2016-10-05,2013-03-10,Port Rogerhaven,Bahamas,8525 Emily Bypass
1,2,F,Rachel,Lynch,2011-11-09,2020-01-04,Thompsonbury,Cook Islands,344 Nicholas Bypass Suite 895
2,3,M,John,Barber,2001-11-05,2017-06-11,North Christine,Turkmenistan,3914 Rodney Crest Apt. 837
3,4,M,David,Lewis,2015-01-22,2006-09-13,Port Anthony,Bouvet Island (Bouvetoya),901 Alexis Hill
4,5,M,Michael,Webster,2015-05-26,2008-05-18,South Miguel,Yemen,74108 Stephanie Shore
...,...,...,...,...,...,...,...,...,...
928,996,F,Theresa,Newman,1996-09-08,2021-06-30,Murphyhaven,Monaco,19698 Mclean Viaduct Apt. 470
929,997,F,Julia,Shaw,2020-05-17,2003-05-14,Dominicmouth,Zimbabwe,78054 Shawn Extension Suite 533
930,998,F,Sherry,Russo,2004-01-21,2001-10-07,West Jamesside,Nepal,7484 Rodriguez Rapid Apt. 845
931,999,F,Mackenzie,Berg,2013-11-29,2003-04-24,Roseburgh,Saint Vincent and the Grenadines,9494 Woods Highway Suite 210


There are other ways to spot duplicates if you are intersted in intermediate aggregations/counts.


In [24]:
# this is equivalent of the countif() in Excal, returns a Series.
df['customer_id'].value_counts().sort_values(ascending=False)[lambda x : x>1].reset_index(name="counts")

,index,counts


In [25]:
# you can use the "unique" method from pandas, but note that it returns a Series object
len(df.customer_id.unique())

933

b) use the groupby commands

In [26]:
df[['customer_id','first_name']].groupby('customer_id').agg('count').reset_index().sort_values("first_name", ascending=False)

,customer_id,first_name
0,1,1
627,665,1
615,653,1
616,654,1
617,655,1
...,...,...
315,329,1
316,330,1
317,331,1
318,332,1


In [27]:
# this may not work in every version of Pandas, to check!
#df[['customer_id','lifetime_id']].groupby(['customer_id','lifetime_id']).agg({'col1':{'customer_id':'count'}}).reset_index()

In [28]:
#ADVANCED WARNING: use the set of unique keys and joining (concat in pandas) with the rest having been converted into a dictionary beforehand. 
# I ve seen it in StackOverflow and could be a faster, anyway it is more convenient than using groupby when you want to just collapse duplicate keys.
# This approach will return an arbitrary value of other fields for each key (ie if you have three duplicated keys and collapse using the df.to_dict it may not bring the values from the last record found, afaik)
pd.concat({k: pd.Series(list(set(v))) for k, v in df.to_dict('list').items()}, axis=1)

,customer_id,gender,first_name,last_name,dob,date_enrollment,city,country,residential address
0,1,M,Gregory,Livingston,1985-06-17,2020-01-04,Jennaview,Netherlands,27380 Tiffany Land Suite 209
1,2,F,Peter,Diaz,1994-03-09,2012-02-01,South Destinyfurt,Yemen,25724 Houston Point Suite 932
2,3,NaN,Yvette,Dorsey,1996-12-29,2012-08-25,Lake Jason,Saint Helena,8370 Matthew Viaduct Suite 200
3,4,NaN,Leonard,Freeman,1980-09-11,2012-07-13,Port Jacobfurt,Cote d'Ivoire,1652 Emily Walks
4,5,NaN,Brett,Leonard,2012-10-01,2012-05-04,Booneland,Albania,463 Colleen Burgs
...,...,...,...,...,...,...,...,...,...
928,996,NaN,NaN,NaN,NaT,NaT,NaN,NaN,35771 Baker Brook
929,997,NaN,NaN,NaN,NaT,NaT,NaN,NaN,187 Jillian Fort Suite 113
930,998,NaN,NaN,NaN,NaT,NaT,NaN,NaN,3908 Douglas Parks Apt. 684
931,999,NaN,NaN,NaN,NaT,NaT,NaN,NaN,4553 Wendy Viaduct


If you are only intested in the number of unique values this way is fast too.  
You can then compare to len(df['customer_id']) and various checks (beware of nan values)

In [29]:
len(set(df['customer_id']))

933

There is a very useful module in python called pandas-profiling that takes a dataframe and does all sorts of analysis, including duplicates and renders it in a beautiful html interactive format.

It doesnt come in the standard library and you need to get it via:

conda install -c conda-forge pandas-profiling

In [1]:
# conda install -c conda-forge pandas-profiling
# needs also tqdm library for rendering progress bars
try:
    from pandas_profiling import ProfileReport
    profile = ProfileReport(df, title="Profiling Report")
    profile.to_file("./output/profiling report.html")
except ModuleNotFoundError as err:
    # Error handling
    print(err)



No module named 'tqdm'
